In [1]:
# df_stat2 = pd.read_csv(r"C:/Users/HELENE/OneDrive/_WCS_/WCS_Projets/Projet_3/Fichiers/df_stat2.csv", low_memory=False)
import pandas as pd
df_attract = pd.read_csv(r"dataset\df_attract.csv")

df_attract['code_departement'] = df_attract['code_departement'].apply(lambda x: str(int(x)) if pd.notnull(x) else '')
df_attract['code_departement'] = df_attract['code_departement'].replace({
    '201': '2A',
    '202': '2B'
})
def format_departement(code):
    if code in ['2A', '2B']:
        return code
    return code.zfill(2)
df_attract['code_departement'] = df_attract['code_departement'].apply(format_departement)


In [2]:
''' fichier STAT2 - 1ere version => pour df_merged '''
print("fichier attract__ \n")
df_attract

fichier attract__ 



,code_departement,valeur_fonciere_x,prix_m2_x,Revenu_médian_2020_x,Population_2021_x,Population_2070_x,Evolution_x,Evolution_pourcentage_x,rang_revenu_x,rang_prix_x,...,Revenu_médian_2020_y,Population_2021_y,Population_2070_y,Evolution_y,Evolution_pourcentage_y,rang_revenu_y,rang_prix_y,rang_evolution_y,note_attractivite_app,note_attractivite_moyenne
0,63,172679.33,1781.38,22510.0,662285.0,719000.0,56715.0,8.56,23.0,37.0,...,22510.0,662285.0,719000.0,56715.0,8.56,23.0,45.0,12.0,26.666667,25.333333
1,01,300623.35,2667.16,24030.0,663202.0,754000.0,90798.0,13.69,6.0,66.0,...,24030.0,663202.0,754000.0,90798.0,13.69,6.0,64.0,6.0,25.333333,25.666667
2,31,302863.71,2808.72,23730.0,1434367.0,1757000.0,322633.0,22.49,8.0,71.0,...,23730.0,1434367.0,1757000.0,322633.0,22.49,8.0,70.0,4.0,27.333333,27.500000
3,35,267929.98,2532.70,22840.0,1098325.0,1291000.0,192675.0,17.54,17.0,63.0,...,22840.0,1098325.0,1291000.0,192675.0,17.54,17.0,79.0,5.0,33.666667,31.000000
4,44,310734.24,3128.23,23430.0,1457806.0,1819000.0,361194.0,24.78,13.0,80.0,...,23430.0,1457806.0,1819000.0,361194.0,24.78,13.0,82.0,1.0,32.000000,31.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,93,394021.24,4747.59,18470.0,1668670.0,1703000.0,34330.0,2.06,93.0,90.0,...,18470.0,1668670.0,1703000.0,34330.0,2.06,93.0,93.0,29.0,71.666667,71.166667
93,971,298436.87,3250.25,NaN,NaN,NaN,NaN,NaN,NaN,81.0,...,NaN,NaN,NaN,NaN,NaN,NaN,72.0,NaN,NaN,NaN
94,972,267796.08,2660.26,NaN,NaN,NaN,NaN,NaN,NaN,65.0,...,NaN,NaN,NaN,NaN,NaN,NaN,68.0,NaN,NaN,NaN
95,973,302407.40,2765.21,NaN,NaN,NaN,NaN,NaN,NaN,69.0,...,NaN,NaN,NaN,NaN,NaN,NaN,63.0,NaN,NaN,NaN


In [3]:
# Affiche le fichier departements.geoson
import geopandas as gpd
gdf_departements = gpd.read_file(r"dataset\departements.geojson")
print("\nGeoDataFrame des départements (extrait):")
print(gdf_departements.head())


GeoDataFrame des départements (extrait):
  code           nom                                           geometry
0   02         Aisne  POLYGON ((3.1727 50.012, 3.1822 50.01234, 3.21...
1   10          Aube  POLYGON ((3.41479 48.39027, 3.42208 48.41334, ...
2   14      Calvados  POLYGON ((-1.11962 49.35557, -1.11503 49.3624,...
3   15        Cantal  POLYGON ((2.50841 45.4785, 2.52444 45.4807, 2....
4   28  Eure-et-Loir  POLYGON ((0.81482 48.67016, 0.82767 48.68072, ...


<font color ="cyan">FOLIUM

<font color="YELLOW">DEPARTEMENT -- CARTE MAISONS ET APPS

In [5]:
import pandas as pd
import geopandas
import folium
from folium.features import GeoJsonTooltip
from branca.colormap import StepColormap  # Ajout pour la colormap personnalisée

# => fichier de données
df_attract

# Assurez-vous que 'code_geo' est de type chaîne de caractères (string)
df_attract['code_departement'] = df_attract['code_departement'].astype(str)

# Ajoute les codes départementaux 2A et 2B pour la Corse
# df_attract['code_departement'] = df_attract['code_departement'].replace({'201': '2A','202': '2B'},inplace=True)
 
# --- 2. Chargement du fichier GeoJSON des départements français ---
try:
    gdf_departements = geopandas.read_file(r"dataset\departements.geojson")

    print("\nFichier GeoJSON 'departements-france.geojson' chargé.")
    print("Premières lignes du GeoDataFrame (extrait):")
    print(gdf_departements.head())
    print("Colonnes disponibles dans le GeoDataFrame (pour identifier le code départemental):")
    print(gdf_departements.columns)

    geojson_code_col = 'code'  # Modifier si nécessaire selon le GeoJSON
    gdf_departements[geojson_code_col] = gdf_departements[geojson_code_col].astype(str)

except FileNotFoundError:
    print("\nERREUR: Le fichier 'departements-france.geojson' n'a pas été trouvé.")
    print("Veuillez le placer dans le dossier 'dataset'.")
    exit()

# --- 3. Fusion des données ---
gdf_merged = gdf_departements.merge(df_attract,
                                    left_on=geojson_code_col,
                                    right_on='code_departement',
                                    how='left')

gdf_merged['note_attractivite_moyenne'] = gdf_merged['note_attractivite_moyenne'].fillna(0)

print("\nGeoDataFrame fusionné (extrait):")
print(gdf_merged[[geojson_code_col, 'note_attractivite_moyenne']].head())

# --- 4. Création de la carte ---
center_lat = gdf_merged.geometry.centroid.y.mean()
center_lon = gdf_merged.geometry.centroid.x.mean()

m = folium.Map(location=[center_lat, center_lon], zoom_start=6)

# Style au survol
def highlight_style_function(x):
    return {
        'fillColor': '#ffffff',
        'color': '#000000',
        'weight': 3,
        'dashArray': '5, 5'
    }

# Colormap personnalisée
colormap = StepColormap(
    colors=['#808080', '#008000', '#FFA500', '#FF0000'],  # gris, vert, orange, rouge
    index=[0, 1, 34.6, 50, 75],
    vmin=0,
    vmax=75
)
colormap.caption = 'Note attractivité par département'
colormap.add_to(m)

# Couche GeoJson avec style basé sur colormap
folium.GeoJson(
    gdf_merged,
    name='Note attractivité par département',
    style_function=lambda feature: {
        'fillColor': colormap(feature['properties']['note_attractivite_moyenne']),
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.7,
    },
    highlight_function=highlight_style_function,
    tooltip=GeoJsonTooltip(
        fields=['nom', 'note_attractivite_moyenne'],
        aliases=['Département :', 'Note attractivité par département :'],
        localize=True,
        sticky=False,
        labels=True,
        style="""
            background-color: #F0F0F0;
            color: #333333;
            font-family: sans-serif;
            font-size: 12px;
            padding: 10px;
        """
    )
).add_to(m)

# Contrôle de couches
folium.LayerControl().add_to(m)

# Sauvegarde
output_file = "carte_attract_globale.html"
m.save(output_file)

print(f"\nCarte interactive enregistrée sous : {output_file}")
print("Ouvrez ce fichier dans votre navigateur web pour visualiser la carte.")



Fichier GeoJSON 'departements-france.geojson' chargé.
Premières lignes du GeoDataFrame (extrait):
  code           nom                                           geometry
0   02         Aisne  POLYGON ((3.1727 50.012, 3.1822 50.01234, 3.21...
1   10          Aube  POLYGON ((3.41479 48.39027, 3.42208 48.41334, ...
2   14      Calvados  POLYGON ((-1.11962 49.35557, -1.11503 49.3624,...
3   15        Cantal  POLYGON ((2.50841 45.4785, 2.52444 45.4807, 2....
4   28  Eure-et-Loir  POLYGON ((0.81482 48.67016, 0.82767 48.68072, ...
Colonnes disponibles dans le GeoDataFrame (pour identifier le code départemental):
Index(['code', 'nom', 'geometry'], dtype='object')

GeoDataFrame fusionné (extrait):
  code  note_attractivite_moyenne
0   02                  62.666667
1   10                  52.500000
2   14                  54.750000
3   15                  46.500000
4   28                  46.500000

Carte interactive enregistrée sous : carte_attract_globale.html
Ouvrez ce fichier dans votre nav

C:\Users\antho\AppData\Local\Temp\ipykernel_25800\1171697152.py:46: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center_lat = gdf_merged.geometry.centroid.y.mean()
C:\Users\antho\AppData\Local\Temp\ipykernel_25800\1171697152.py:47: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center_lon = gdf_merged.geometry.centroid.x.mean()


<font color="YELLOW">DEPARTEMENT -- CARTE MAISONS

In [6]:
import pandas as pd
import geopandas
import folium
from folium.features import GeoJsonTooltip
from branca.colormap import StepColormap  # Ajout pour la colormap personnalisée

# => fichier de données
df_attract

# Assurez-vous que 'code_geo' est de type chaîne de caractères (string)
df_attract['code_departement'] = df_attract['code_departement'].astype(str)

# Ajoute les codes départementaux 2A et 2B pour la Corse
# df_attract['code_departement'] = df_attract['code_departement'].replace({'201': '2A','202': '2B'},inplace=True)

# --- 2. Chargement du fichier GeoJSON des départements français ---
try:
    gdf_departements = geopandas.read_file(r"dataset\departements.geojson")

    print("\nFichier GeoJSON 'departements-france.geojson' chargé.")
    print("Premières lignes du GeoDataFrame (extrait):")
    print(gdf_departements.head())
    print("Colonnes disponibles dans le GeoDataFrame (pour identifier le code départemental):")
    print(gdf_departements.columns)

    geojson_code_col = 'code'  # Modifier si nécessaire selon le GeoJSON
    gdf_departements[geojson_code_col] = gdf_departements[geojson_code_col].astype(str)

except FileNotFoundError:
    print("\nERREUR: Le fichier 'departements-france.geojson' n'a pas été trouvé.")
    print("Veuillez le placer dans le dossier 'dataset'.")
    exit()

# --- 3. Fusion des données ---
gdf_merged = gdf_departements.merge(df_attract,
                                    left_on=geojson_code_col,
                                    right_on='code_departement',
                                    how='left')

gdf_merged['note_attractivite_maison'] = gdf_merged['note_attractivite_maison'].fillna(0)

print("\nGeoDataFrame fusionné (extrait):")
print(gdf_merged[[geojson_code_col, 'note_attractivite_maison']].head())

# --- 4. Création de la carte ---
center_lat = gdf_merged.geometry.centroid.y.mean()
center_lon = gdf_merged.geometry.centroid.x.mean()

m = folium.Map(location=[center_lat, center_lon], zoom_start=6)

# Style au survol
def highlight_style_function(x):
    return {
        'fillColor': '#ffffff',
        'color': '#000000',
        'weight': 3,
        'dashArray': '5, 5'
    }

# Colormap personnalisée
colormap = StepColormap(
    colors=['#808080', '#008000', '#FFA500', '#FF0000'],  # gris, vert, orange, rouge
    index=[0, 1, 34.6, 50, 75],
    vmin=0,
    vmax=75
)
colormap.caption = 'Note attractivité si type de logement = maison'
colormap.add_to(m)

# Couche GeoJson avec style basé sur colormap
folium.GeoJson(
    gdf_merged,
    name='Note attractivité par département',
    style_function=lambda feature: {
        'fillColor': colormap(feature['properties']['note_attractivite_maison']),
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.7,
    },
    highlight_function=highlight_style_function,
    tooltip=GeoJsonTooltip(
        fields=['nom', 'note_attractivite_maison','valeur_fonciere_x', 'prix_m2_x', 'rang_prix_x', 'Revenu_médian_2020_x', 'rang_revenu_x','Evolution_pourcentage_x', 'rang_evolution_x'],
        aliases=['Département :', 'Note attractivité :', 'Valeur foncière moyenne :', 'Prix m² moyen :', 'Rang prix m² :', 'Revenu médian :', 'Rang revenu :', 'Evolution population 2070 (%) :', 'Rang evolution :'],
        localize=True,
        sticky=False,
        labels=True,
        style="""
            background-color: #F0F0F0;
            color: #333333;
            font-family: sans-serif;
            font-size: 12px;
            padding: 10px;
        """
    )
).add_to(m)

# Contrôle de couches
folium.LayerControl().add_to(m)

# Sauvegarde
output_file = "carte_attract_maison.html"
m.save(output_file)

print(f"\nCarte interactive enregistrée sous : {output_file}")
print("Ouvrez ce fichier dans votre navigateur web pour visualiser la carte.")



Fichier GeoJSON 'departements-france.geojson' chargé.
Premières lignes du GeoDataFrame (extrait):
  code           nom                                           geometry
0   02         Aisne  POLYGON ((3.1727 50.012, 3.1822 50.01234, 3.21...
1   10          Aube  POLYGON ((3.41479 48.39027, 3.42208 48.41334, ...
2   14      Calvados  POLYGON ((-1.11962 49.35557, -1.11503 49.3624,...
3   15        Cantal  POLYGON ((2.50841 45.4785, 2.52444 45.4807, 2....
4   28  Eure-et-Loir  POLYGON ((0.81482 48.67016, 0.82767 48.68072, ...
Colonnes disponibles dans le GeoDataFrame (pour identifier le code départemental):
Index(['code', 'nom', 'geometry'], dtype='object')

GeoDataFrame fusionné (extrait):
  code  note_attractivite_maison
0   02                 60.666667
1   10                 51.333333
2   14                 52.666667
3   15                 46.000000
4   28                 46.833333

Carte interactive enregistrée sous : carte_attract_maison.html
Ouvrez ce fichier dans votre navigateur

C:\Users\antho\AppData\Local\Temp\ipykernel_25800\3655166271.py:46: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center_lat = gdf_merged.geometry.centroid.y.mean()
C:\Users\antho\AppData\Local\Temp\ipykernel_25800\3655166271.py:47: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center_lon = gdf_merged.geometry.centroid.x.mean()


<font color="YELLOW">DEPARTEMENT -- CARTE APPARTEMENTS

In [ ]:
import pandas as pd
import geopandas
import folium
from folium.features import GeoJsonTooltip
from branca.colormap import StepColormap  # Ajout pour la colormap personnalisée

# => fichier de données
df_attract

# Assurez-vous que 'code_geo' est de type chaîne de caractères (string)
df_attract['code_departement'] = df_attract['code_departement'].astype(str)

# Ajoute les codes départementaux 2A et 2B pour la Corse
# df_attract['code_departement'] = df_attract['code_departement'].replace({'201': '2A','202': '2B'},inplace=True)

# --- 2. Chargement du fichier GeoJSON des départements français ---
try:
    gdf_departements = geopandas.read_file(r"dataset\departements.geojson")

    print("\nFichier GeoJSON 'departements-france.geojson' chargé.")
    print("Premières lignes du GeoDataFrame (extrait):")
    print(gdf_departements.head())
    print("Colonnes disponibles dans le GeoDataFrame (pour identifier le code départemental):")
    print(gdf_departements.columns)

    geojson_code_col = 'code'  # Modifier si nécessaire selon le GeoJSON
    gdf_departements[geojson_code_col] = gdf_departements[geojson_code_col].astype(str)

except FileNotFoundError:
    print("\nERREUR: Le fichier 'departements-france.geojson' n'a pas été trouvé.")
    print("Veuillez le placer dans le dossier 'dataset'.")
    exit()

# --- 3. Fusion des données ---
gdf_merged = gdf_departements.merge(df_attract,
                                    left_on=geojson_code_col,
                                    right_on='code_departement',
                                    how='left')

gdf_merged['note_attractivite_app'] = gdf_merged['note_attractivite_app'].fillna(0)

print("\nGeoDataFrame fusionné (extrait):")
print(gdf_merged[[geojson_code_col, 'note_attractivite_app']].head())

# --- 4. Création de la carte ---
center_lat = gdf_merged.geometry.centroid.y.mean()
center_lon = gdf_merged.geometry.centroid.x.mean()

m = folium.Map(location=[center_lat, center_lon], zoom_start=6)

# Style au survol
def highlight_style_function(x):
    return {
        'fillColor': '#ffffff',
        'color': '#000000',
        'weight': 3,
        'dashArray': '5, 5'
    }

# Colormap personnalisée
colormap = StepColormap(
    colors=['#808080', '#008000', '#FFA500', '#FF0000'],  # gris, vert, orange, rouge
    index=[0, 1, 34.6, 50, 75],
    vmin=0,
    vmax=75
)
colormap.caption = 'Note attractivité si type de logement = appartement'
colormap.add_to(m)

# Couche GeoJson avec style basé sur colormap
folium.GeoJson(
    gdf_merged,
    name='Note attractivité par département',
    style_function=lambda feature: {
        'fillColor': colormap(feature['properties']['note_attractivite_app']),
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.7,
    },
    highlight_function=highlight_style_function,
    tooltip=GeoJsonTooltip(
        fields=['nom', 'note_attractivite_app','valeur_fonciere_y', 'prix_m2_y', 'rang_prix_y', 'Revenu_médian_2020_y', 'rang_revenu_y', 'Evolution_pourcentage_y', 'rang_evolution_y'],
        aliases=['Département :', 'Note attractivité :', 'Valeur foncière moyenne :', 'Prix m² moyen :', 'Rang prix m² :', 'Revenu médian :', 'Rang revenu :', 'Evolution population 2070 (%) :', 'Rang evolution :'],
        localize=True,
        sticky=False,
        labels=True,
        style="""
            background-color: #F0F0F0;
            color: #333333;
            font-family: sans-serif;
            font-size: 12px;
            padding: 10px;
        """
    )
).add_to(m)

# Contrôle de couches
folium.LayerControl().add_to(m)

# Sauvegarde
output_file = "carte_attract_app.html"
m.save(output_file)

print(f"\nCarte interactive enregistrée sous : {output_file}")
print("Ouvrez ce fichier dans votre navigateur web pour visualiser la carte.")



Fichier GeoJSON 'departements-france.geojson' chargé.
Premières lignes du GeoDataFrame (extrait):
  code           nom                                           geometry
0   02         Aisne  POLYGON ((3.1727 50.012, 3.1822 50.01234, 3.21...
1   10          Aube  POLYGON ((3.41479 48.39027, 3.42208 48.41334, ...
2   14      Calvados  POLYGON ((-1.11962 49.35557, -1.11503 49.3624,...
3   15        Cantal  POLYGON ((2.50841 45.4785, 2.52444 45.4807, 2....
4   28  Eure-et-Loir  POLYGON ((0.81482 48.67016, 0.82767 48.68072, ...
Colonnes disponibles dans le GeoDataFrame (pour identifier le code départemental):
Index(['code', 'nom', 'geometry'], dtype='object')

GeoDataFrame fusionné (extrait):
  code  note_attractivite_app
0   02              64.666667
1   10              53.666667
2   14              56.833333
3   15              47.000000
4   28              46.166667


C:\Users\antho\AppData\Local\Temp\ipykernel_25800\4173100047.py:46: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center_lat = gdf_merged.geometry.centroid.y.mean()
C:\Users\antho\AppData\Local\Temp\ipykernel_25800\4173100047.py:47: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center_lon = gdf_merged.geometry.centroid.x.mean()



Carte interactive enregistrée sous : carte_attract_app.html
Ouvrez ce fichier dans votre navigateur web pour visualiser la carte.


: 